In [1]:
# import package
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.cluster import DBSCAN, KMeans
import hdbscan
from yellowbrick.cluster import SilhouetteVisualizer
import skfuzzy as fuzz
import skfda
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA


## Import data

In [2]:
DIR = "../../data/"
SUBWAYUS = "Subway USA/subway_usa_"
train_df = pd.read_csv(DIR + SUBWAYUS + "processed_train.csv", index_col="store")
test_df = pd.read_csv(DIR + SUBWAYUS + "processed_test.csv", index_col="store")
merged_df = pd.concat([train_df, test_df])
stores = pd.read_csv(DIR + SUBWAYUS + "stores.csv", index_col="store")

## PCA

In [3]:
pca = PCA(n_components=37, whiten=True, random_state=42)
pca.fit(train_df)

PCA(n_components=37, random_state=42, whiten=True)

In [4]:
transformed_features = pd.DataFrame(
    abs(pca.components_), 
    columns=train_df.columns.tolist(), 
    index=pca.get_feature_names_out(train_df.columns.tolist()))
transformed_features.head()

,gq_college_p_ta,hhinc30lt_p_ta,crime_total_index_ta,genz_p_ta,osm_nearest_exit_dist,pop_migration_ta,hh_type_male_child_p_ta,nces_public_schools_nearest_dist,dtpop_work_at_home_p_ta,dtpop_unemployed_p_ta,...,popgr10cn_ta,com12pl_p_ta,ipeds_postsecondary_schools_5mi,dmm_count_1mi,pop_transient_ta,osm_highway_exits_count_5mi,dmm_nearest_dist,spend_foodbev_ta,market_size,store_density
pca0,0.032143,0.029926,0.052401,0.025407,0.054703,0.085938,0.024785,0.041600,0.017302,0.024829,...,0.001774,0.060314,0.142424,0.071491,0.066374,0.129280,0.071857,0.146144,0.196444,0.184512
pca1,0.044231,0.209281,0.112004,0.033100,0.032024,0.060407,0.062229,0.016747,0.171551,0.020595,...,0.062714,0.048216,0.037714,0.007242,0.016849,0.022710,0.044136,0.009138,0.173871,0.026345
pca2,0.072883,0.009241,0.010677,0.052873,0.054191,0.012310,0.111821,0.031418,0.016301,0.154527,...,0.056976,0.070223,0.038408,0.048367,0.090686,0.029572,0.060892,0.081892,0.205584,0.027271
pca3,0.130971,0.008827,0.168812,0.149642,0.106442,0.142927,0.012485,0.024135,0.030017,0.147050,...,0.050848,0.084003,0.084570,0.080869,0.037755,0.020982,0.128720,0.067819,0.149431,0.107449
pca4,0.060645,0.009647,0.005581,0.137944,0.091857,0.018326,0.023740,0.015704,0.008094,0.009498,...,0.132774,0.084595,0.061621,0.035936,0.086095,0.059558,0.126236,0.030647,0.255061,0.091413


#### Filter and Count Important Features (of heavy weights)

Filter the values in the PCA component matrix, and count the occurrence of the features in the filtered matrix.

In [5]:
W = abs(pca.components_)
features = transformed_features.columns
long_results = []
for i in range(W.shape[0]):
    array = W[i]
    heavy_idx = np.where(array > 0.25)
    long_results += list(features[heavy_idx])

In [6]:
count = pd.Series(long_results).value_counts()
pca_features = count.index.tolist()
count

nces_public_schools_nearest_dist                    6
other_p_ta                                          5
popgr10cn_ta                                        5
gq_other_p_ta                                       5
inrix_total_ta                                      4
osm_highway_exits_count_1mi                         4
market_size                                         4
hrsa_hospitals_nearest_dist                         3
occ_unclassified_p_ta                               3
hh_type_male_child_p_ta                             3
transitstop_nearest_dist                            3
ipeds_postsecondary_schools_total_enrollment_1mi    3
hh_type_male_nochild_p_ta                           2
nces_private_schools_1mi                            2
nces_private_schools_total_enrollment_1mi           2
hrsa_number_of_certified_beds_1mi                   2
hrsa_hospitals_1mi                                  2
genz_p_ta                                           2
dtpop_homemakers_p_ta       

In [7]:
pca_feature_weight = pd.Series(np.sum(W, axis=0), index=transformed_features.columns)
pca_feature_weight.sort_values(ascending=False, inplace=True)
pca_feature_weight = pca_feature_weight.filter(items = pca_features, axis=0)
pca_feature_weight

nces_public_schools_nearest_dist                    4.267739
other_p_ta                                          4.095563
popgr10cn_ta                                        3.869590
gq_other_p_ta                                       4.151083
inrix_total_ta                                      4.105526
osm_highway_exits_count_1mi                         3.838588
market_size                                         4.839649
hrsa_hospitals_nearest_dist                         3.772741
occ_unclassified_p_ta                               4.241153
hh_type_male_child_p_ta                             3.802947
transitstop_nearest_dist                            3.879235
ipeds_postsecondary_schools_total_enrollment_1mi    3.639659
hh_type_male_nochild_p_ta                           3.464510
nces_private_schools_1mi                            2.881414
nces_private_schools_total_enrollment_1mi           3.166516
hrsa_number_of_certified_beds_1mi                   3.009360
hrsa_hospitals_1mi      

In [8]:
important_features = pca_feature_weight.sort_values(ascending=False)[:40].index.tolist()

In [9]:
reduced_train = train_df[pca_features]
reduced_test = test_df[pca_features]
reduced_train

,nces_public_schools_nearest_dist,other_p_ta,popgr10cn_ta,gq_other_p_ta,inrix_total_ta,osm_highway_exits_count_1mi,market_size,hrsa_hospitals_nearest_dist,occ_unclassified_p_ta,hh_type_male_child_p_ta,...,centerxy_gla_effective_5mi,com0508_p_ta,hh_type_nonfam_p_ta,dmm_count_1mi,dmm_gla_1mi,com0205_p_ta,dtpop_students_p_ta,dtpop_students_post_secondary_p_ta,hhgrpycy_ta,nces_private_schools_nearest_dist
store,,,,,,,,,,,,,,,,,,,,,
6150300,-0.719608,-0.352255,-0.248036,2.510505,-0.587414,-0.034179,2.0,-0.481310,-0.408704,1.104476,...,-0.659887,-0.701691,-1.109287,-0.462636,-0.408427,-1.195003,-0.116104,-0.247143,-0.623647,-0.502267
3784100,0.703190,0.213035,0.285237,1.338613,1.135356,-0.294265,2.0,-0.060964,-0.319611,-0.646358,...,0.346080,0.636829,-0.371554,0.637963,1.359225,0.433995,-0.096388,-0.163931,1.015249,-0.398822
1192500,-0.398347,0.068600,-0.241841,-0.744864,-0.593298,-0.554352,2.0,-0.631452,0.045673,-0.410199,...,-0.346036,-0.445058,0.366179,0.637963,1.139668,-0.110564,-0.535503,-0.514482,-0.346789,-0.465102
449400,-0.533603,-0.399570,-0.117522,-0.151132,-0.999861,-0.554352,4.0,-0.705955,-0.141424,0.208701,...,-0.498695,-0.295355,0.143518,0.637963,0.463615,0.006395,-0.302504,-0.287863,-0.194876,-0.389734
2292700,-0.303830,-0.250154,-0.187331,-0.471355,2.300364,-0.554352,0.0,-0.654793,-0.399795,-0.304334,...,0.834463,-0.715530,-0.645186,-0.462636,-0.408427,-1.051846,0.337349,0.301700,-0.357065,-0.462221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440100,-0.805969,-0.334823,-0.299179,-0.682918,-0.134447,-0.034179,4.0,-0.070710,-0.435432,-0.336908,...,-0.540838,-0.699175,-0.103288,-0.462636,-0.408427,0.826040,-0.562388,-0.542810,-1.169666,-0.486132
6487000,0.241304,-0.299959,0.038132,-0.743172,0.289174,0.225908,4.0,-0.440797,-0.257245,0.404143,...,0.085553,0.394034,-0.578119,-0.462636,-0.408427,1.732703,0.498657,0.558417,0.035422,-0.230307
2493300,0.398550,-0.302450,-0.235070,-0.614203,-0.142035,-0.554352,1.0,-0.401734,-0.301792,-0.157753,...,0.113689,0.626765,1.028797,-0.462636,-0.408427,0.701596,-0.558803,-0.544580,-0.349008,-0.393638


In [10]:
def corr_pair(target_corr, corr_threshold=0.6):
    np.fill_diagonal(target_corr.values, 0)
    sorted_pair = target_corr.abs().unstack().sort_values(kind="quicksort", ascending=False)
    return sorted_pair[sorted_pair > corr_threshold]

correlated_pairs = corr_pair(reduced_train.corr())

In [11]:
correlated_pairs[:20:2]

pop5y_cagr_ta              popgrpy_ta                                   0.999096
popgrpy_ta                 hhgrpycy_ta                                  0.997022
dtpop_students_p_ta        dtpop_students_post_secondary_p_ta           0.996903
hhgrpycy_ta                pop5y_cagr_ta                                0.996545
dmm_count_1mi              dmm_gla_1mi                                  0.887327
nces_private_schools_1mi   nces_private_schools_total_enrollment_1mi    0.822156
com12pl_p_ta               com0205_p_ta                                 0.820206
hh_type_male_nochild_p_ta  hh_type_male_p_ta                            0.814916
hrsa_hospitals_1mi         hrsa_number_of_certified_beds_1mi            0.794156
dmm_gla_1mi                centerxy_gla_effective_1mi                   0.778494
dtype: float64

In [12]:
def corr_pair_drop(feature_pairs, corr_with_target):
    selected, discarded = [], []
    for f1, f2 in feature_pairs:
        if abs(corr_with_target[f1]) < abs(corr_with_target[f2]):
            selected.append(f2)
            discarded.append(f1)
        else:
            selected.append(f1)
            discarded.append(f2)
    final_discarded = set(discarded) - set(selected) 
    return list(final_discarded)

In [13]:
corr_drop_list = corr_pair_drop(correlated_pairs.index, pca_feature_weight)

In [14]:
important_features = list(set(important_features) - set(corr_drop_list))

In [15]:
reduced_train.drop(columns=corr_drop_list, inplace=True)
reduced_test.drop(columns=corr_drop_list, inplace=True)

/var/folders/gy/t0bm3n_d2g5grs1b9jx68r680000gn/T/ipykernel_29349/1625573170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_train.drop(columns=corr_drop_list, inplace=True)
/var/folders/gy/t0bm3n_d2g5grs1b9jx68r680000gn/T/ipykernel_29349/1625573170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_test.drop(columns=corr_drop_list, inplace=True)


In [16]:
correlated_pairs = corr_pair(reduced_train.corr())
correlated_pairs[:20:2]

hhgrpycy_ta  pop5y_cagr_ta    0.996545
dmm_gla_1mi  dmm_count_1mi    0.887327
dtype: float64

### PCA model (90% Variance) - Layer 2

In [17]:
pca_2 = PCA(n_components=34, whiten=True, random_state=42)
pca_2.fit(reduced_train)

PCA(n_components=34, random_state=42, whiten=True)

In [18]:
transformed_features = pd.DataFrame(
    abs(pca_2.components_), 
    columns=reduced_train.columns.tolist(), 
    index=pca_2.get_feature_names_out(reduced_train.columns.tolist()))
transformed_features.head()

,nces_public_schools_nearest_dist,other_p_ta,popgr10cn_ta,gq_other_p_ta,inrix_total_ta,osm_highway_exits_count_1mi,market_size,hrsa_hospitals_nearest_dist,occ_unclassified_p_ta,hh_type_male_child_p_ta,...,com0811_p_ta,centerxy_gla_effective_5mi,com0508_p_ta,hh_type_nonfam_p_ta,dmm_count_1mi,dmm_gla_1mi,com0205_p_ta,dtpop_students_p_ta,hhgrpycy_ta,nces_private_schools_nearest_dist
pca0,0.084450,0.001250,0.033643,0.098897,0.172310,0.150195,0.672569,0.145876,0.107750,0.041902,...,0.129547,0.253455,0.139704,0.083366,0.147097,0.138440,0.059567,0.007099,0.067144,0.199521
pca1,0.049990,0.026608,0.130747,0.117017,0.045925,0.154009,0.276970,0.159046,0.069946,0.148633,...,0.157342,0.055231,0.022587,0.295706,0.259308,0.222818,0.236913,0.127702,0.141096,0.026399
pca2,0.160463,0.006152,0.184515,0.028682,0.070758,0.029916,0.040794,0.149841,0.080173,0.218818,...,0.055385,0.048752,0.057632,0.091689,0.116430,0.109961,0.140268,0.212497,0.281216,0.058744
pca3,0.026914,0.078148,0.272884,0.031853,0.030475,0.054269,0.049920,0.022103,0.032748,0.071280,...,0.181200,0.054578,0.234164,0.142393,0.077242,0.067165,0.114749,0.286410,0.436375,0.048361
pca4,0.005122,0.069404,0.080562,0.172494,0.061389,0.095708,0.104862,0.071782,0.069264,0.190405,...,0.152745,0.029876,0.200461,0.231571,0.240287,0.241483,0.164868,0.310893,0.069773,0.160344


#### Filter and Count Important Features (of heavy weights)

Filter the values in the PCA component matrix, and count the occurrence of the features in the filtered matrix.

In [19]:
W = abs(pca_2.components_)
features = transformed_features.columns
long_results = []
for i in range(W.shape[0]):
    array = W[i]
    heavy_idx = np.where(array > 0.15)
    long_results += list(features[heavy_idx])

In [20]:
count = pd.Series(long_results).value_counts()
count

pop_seasonal_ta                                     16
asian_p_ta                                          15
transitstop_nearest_dist                            14
com0811_p_ta                                        14
nces_private_schools_total_enrollment_1mi           14
dtpop_homemakers_p_ta                               14
black_p_ta                                          13
osm_nearest_exit_dist                               13
pop_transient_ta                                    13
occ_bc_p_ta                                         12
mortgage_avgrisk_ta                                 12
hrsa_hospitals_nearest_dist                         12
hrsa_number_of_certified_beds_1mi                   12
inrix_total_ta                                      11
hh_type_male_nochild_p_ta                           11
hh_type_male_child_p_ta                             11
hh_type_nonfam_p_ta                                 11
ipeds_postsecondary_schools_total_enrollment_1mi    10
nces_priva

In [21]:
pca_2_features = count.index.tolist()

In [22]:
pca_2_feature_weight = pd.Series(np.sum(W, axis=0), index=transformed_features.columns)
pca_2_feature_weight.sort_values(ascending=False, inplace=True)
pca_2_feature_weight = pca_feature_weight.filter(items=pca_2_features, axis=0)
pca_2_feature_weight

pop_seasonal_ta                                     3.401899
asian_p_ta                                          3.168327
transitstop_nearest_dist                            3.879235
com0811_p_ta                                        3.653263
nces_private_schools_total_enrollment_1mi           3.166516
dtpop_homemakers_p_ta                               3.366399
black_p_ta                                          2.947576
osm_nearest_exit_dist                               3.730653
pop_transient_ta                                    3.288879
occ_bc_p_ta                                         3.379949
mortgage_avgrisk_ta                                 2.433900
hrsa_hospitals_nearest_dist                         3.772741
hrsa_number_of_certified_beds_1mi                   3.009360
inrix_total_ta                                      4.105526
hh_type_male_nochild_p_ta                           3.464510
hh_type_male_child_p_ta                             3.802947
hh_type_nonfam_p_ta     

In [23]:
pca_2_feature_weight = pca_2_feature_weight.sort_values(ascending=False)
important_features_2 = pca_2_feature_weight[:40].index.tolist()

In [24]:
important_features = list(set(important_features + important_features_2))

In [25]:
reduced_train = reduced_train[important_features]
reduced_test = reduced_test[important_features]
reduced_train

,osm_nearest_exit_dist,hh_type_male_nochild_p_ta,dmm_gla_1mi,millenial_p_ta,hhgrpycy_ta,ipeds_postsecondary_schools_total_enrollment_1mi,inrix_total_ta,other_p_ta,hrsa_number_of_certified_beds_1mi,com0811_p_ta,...,mortgage_avgrisk_ta,market_size,pop_transient_ta,centerxy_gla_effective_5mi,hispanic_p_ta,occ_unclassified_p_ta,nces_public_schools_nearest_dist,dtpop_students_p_ta,com0205_p_ta,genz_p_ta
store,,,,,,,,,,,,,,,,,,,,,
6150300,-0.425379,0.567561,-0.408427,1.247750,-0.623647,-0.206017,-0.587414,-0.352255,-0.254175,3.304383,...,-0.694918,2.0,-0.251278,-0.659887,2.798942,-0.408704,-0.719608,-0.116104,-1.195003,0.185124
3784100,-0.360187,0.502734,1.359225,-0.260020,1.015249,-0.206017,1.135356,0.213035,-0.254175,-1.035261,...,0.401362,2.0,-0.251278,0.346080,-0.050598,-0.319611,0.703190,-0.096388,0.433995,-0.207241
1192500,-0.301741,0.106573,1.139668,0.068360,-0.346789,-0.206017,-0.593298,0.068600,-0.254175,-0.493172,...,0.634310,2.0,-0.144587,-0.346036,-0.106979,0.045673,-0.398347,-0.535503,-0.110564,-0.469702
449400,-0.021385,0.517140,0.463615,-0.366396,-0.194876,-0.206017,-0.999861,-0.399570,0.352885,-0.425777,...,-0.478647,4.0,-0.207535,-0.498695,0.224198,-0.141424,-0.533603,-0.302504,0.006395,-0.183381
2292700,-0.185947,1.431912,-0.408427,1.418877,-0.357065,-0.206017,2.300364,-0.250154,0.796798,1.090608,...,2.159798,0.0,-0.234207,0.834463,2.551067,-0.399795,-0.303830,0.337349,-1.051846,0.177171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440100,-0.383242,-0.476864,-0.408427,-1.444035,-1.169666,-0.206017,-0.134447,-0.334823,-0.254175,-0.920983,...,-1.467668,4.0,-0.213936,-0.540838,-0.792189,-0.435432,-0.805969,-0.562388,0.826040,-0.649978
6487000,-0.376830,0.012935,-0.408427,0.188611,0.035422,-0.206017,0.289174,-0.299959,-0.254175,-0.836007,...,-0.369457,4.0,0.274174,0.085553,-0.590537,-0.257245,0.241304,0.498657,1.732703,0.357447
2493300,-0.320988,-0.188747,-0.408427,-0.306270,-0.349008,-0.206017,-0.142035,-0.302450,-0.254175,1.317699,...,0.422427,1.0,-0.183262,0.113689,-0.613902,-0.301792,0.398550,-0.558803,0.701596,-0.649978


In [26]:
def corr_pair(target_corr, corr_threshold=0.6):
    np.fill_diagonal(target_corr.values, 0)
    sorted_pair = target_corr.abs().unstack().sort_values(kind="quicksort", ascending=False)
    return sorted_pair[sorted_pair > corr_threshold]

correlated_pairs = corr_pair(reduced_train.corr())

In [27]:
correlated_pairs[:20:2]

hhgrpycy_ta    pop5y_cagr_ta    0.996545
dmm_count_1mi  dmm_gla_1mi      0.887327
dtype: float64

In [28]:
print(pca_feature_weight['pop5y_cagr_ta'], pca_feature_weight['hhgrpycy_ta'])
print(pca_feature_weight['dmm_gla_1mi'], pca_feature_weight['dmm_count_1mi'])

1.999758470316917 2.002079075081284
2.5224262294073627 2.4200185378338945


In [29]:
corr_drop_list = ['pop5y_cagr_ta', 'dmm_count_1mi']

In [30]:
reduced_train.drop(columns=corr_drop_list, inplace=True)
reduced_test.drop(columns=corr_drop_list, inplace=True)

/var/folders/gy/t0bm3n_d2g5grs1b9jx68r680000gn/T/ipykernel_29349/1625573170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_train.drop(columns=corr_drop_list, inplace=True)


In [31]:
correlated_pairs = corr_pair(reduced_train.corr())
correlated_pairs

Series([], dtype: float64)

In [32]:
train_df = reduced_train
train_df.columns.to_list()

['osm_nearest_exit_dist',
 'hh_type_male_nochild_p_ta',
 'dmm_gla_1mi',
 'millenial_p_ta',
 'hhgrpycy_ta',
 'ipeds_postsecondary_schools_total_enrollment_1mi',
 'inrix_total_ta',
 'other_p_ta',
 'hrsa_number_of_certified_beds_1mi',
 'com0811_p_ta',
 'occ_bc_p_ta',
 'black_p_ta',
 'hh_type_male_child_p_ta',
 'dtpop_homemakers_p_ta',
 'transitstop_nearest_dist',
 'asian_p_ta',
 'popgr10cn_ta',
 'gq_other_p_ta',
 'nces_private_schools_total_enrollment_1mi',
 'hh_type_nonfam_p_ta',
 'osm_highway_exits_count_1mi',
 'hrsa_hospitals_nearest_dist',
 'pop_seasonal_ta',
 'com0508_p_ta',
 'nces_private_schools_nearest_dist',
 'mortgage_avgrisk_ta',
 'market_size',
 'pop_transient_ta',
 'centerxy_gla_effective_5mi',
 'hispanic_p_ta',
 'occ_unclassified_p_ta',
 'nces_public_schools_nearest_dist',
 'dtpop_students_p_ta',
 'com0205_p_ta',
 'genz_p_ta']

## Without PCA

In [33]:
selected_features = ['age0018_p_ta',
 'age65pl_p_ta',
 'age85pl_p_ta',
#  'asian_p_ta',
#  'avg_faminc_ta',
#  'avghhinc_ta',
 'banks_1mi',
 'banks_5mi',
#  'black_p_ta',
#  'boomer_p_ta',
 'centerxy_gla_effective_1mi',
 'centerxy_gla_effective_5mi',
 'com0002_p_ta',
#  'com0205_p_ta',
 'com0508_p_ta',
#  'com0811_p_ta',
 'com12pl_p_ta',
 'crime_total_index_ta',
 'daypop_dens_ta',
 'disposable_inc_avg_ta',
 'dmm_count_1mi',
 'dmm_count_5mi',
 'dmm_gla_1mi',
 'dmm_gla_5mi',
#  'dmm_nearest_dist',
 'dtpop_children_at_home_p_ta',
 'dtpop_homemakers_p_ta',
 'dtpop_retired_disabled_p_ta',
 'dtpop_students_9th_12th_p_ta',
 'dtpop_students_p_ta',
 'dtpop_students_post_secondary_p_ta',
 'dtpop_students_prek_8th_p_ta',
 'dtpop_ta',
 'dtpop_unemployed_p_ta',
 'dtpop_work_at_home_p_ta',
 'edu_bachplus_p_ta',
 'emp_p_ta',
 'empcy_ta',
 'gdp_ta',
#  'genx_p_ta',
#  'genz_p_ta',
#  'gq_college_p_ta',
#  'gq_other_p_ta',
 'hh_dens_ta',
 'hh_expected_pers_ta',
 'hh_expected_vehicle_ta',
#  'hh_inc_gt_500k_p_ta',
#  'hh_inc_gt_75k_p_ta',
#  'hh_inc_lt_75k_p_ta',
 'hh_type_1pers_p_ta',
 'hh_type_fam_p_ta',
#  'hh_type_female_child_p_ta',
#  'hh_type_female_nochild_p_ta',
#  'hh_type_female_p_ta',
#  'hh_type_male_child_p_ta',
#  'hh_type_male_nochild_p_ta',
#  'hh_type_male_p_ta',
#  'hh_type_married_child_p_ta',
#  'hh_type_married_nochild_p_ta',
#  'hh_type_married_p_ta',
#  'hh_type_nonfam_p_ta',
 'hhcy_ta',
#  'hhgrfypy_ta',
#  'hhgrpycy_ta',
 'hhinc100pl_p_ta',
 'hhinc150pl_p_ta',
 'hhinc30lt_p_ta',
#  'hispanic_p_ta',
#  'hrsa_hospitals_1mi',
#  'hrsa_hospitals_5mi',
#  'hrsa_hospitals_nearest_dist',
 'hrsa_number_of_certified_beds_1mi',
 'hrsa_number_of_certified_beds_5mi',
 'hu_ownerocc_ta',
 'hu_renterocc_ta',
 'hu_vacant_ta',
 'inrix_total_ta',
#  'ipeds_postsecondary_nearest_dist',
#  'ipeds_postsecondary_schools_1mi',
#  'ipeds_postsecondary_schools_5mi',
 'ipeds_postsecondary_schools_total_enrollment_1mi',
 'ipeds_postsecondary_schools_total_enrollment_5mi',
 'market_size',
 'medhhinc_ta',
 'medsalcy_ta',
 'millenial_p_ta',
#  'mortgage_avgrisk_ta',
 'nces_private_schools_1mi',
 'nces_private_schools_5mi',
#  'nces_private_schools_nearest_dist',
 'nces_private_schools_total_enrollment_1mi',
 'nces_private_schools_total_enrollment_5mi',
 'nces_public_schools_1mi',
 'nces_public_schools_5mi',
#  'nces_public_schools_nearest_dist',
 'nces_public_schools_total_enrollment_1mi',
 'nces_public_schools_total_enrollment_5mi',
 'occ_bc_p_ta',
#  'occ_unclassified_p_ta',
 'occ_wc_p_ta',
 'occhu_ta',
 'osm_highway_exits_count_1mi',
 'osm_highway_exits_count_5mi',
 'osm_nearest_exit_dist',
#  'other_p_ta',
 'percapita_inc_ta',
 'places_of_worship_1mi',
 'places_of_worship_5mi',
 'pop5y_cagr_ta',
 'pop_dens_ta',
 'pop_migration_ta',
 'pop_seasonal_ta',
 'pop_transient_ta',
 'popcy_ta',
 'popgr10cn_ta',
 'popgrfy_ta',
 'popgrpy_ta',
 'poverty_inpoverty_p_ta',
#  'spend_breakfastbrunch_ta',
#  'spend_dinner_ta',
 'spend_foodbev_ta',
#  'spend_lunch_ta',
 'store_density',
 'transitstop_nearest_dist',
 'transitstops',
 'wealth_hhavg_ta',
#  'wealth_hhtotal_ta',
#  'white_p_ta'
 ]

merged_df = merged_df[selected_features]

## Kmeans

In [34]:
# k = 3
# km = KMeans(n_clusters=k, n_init='auto', random_state=42)
# km.fit(train_df)

# cluster_labels = km.labels_
# counts = np.bincount(cluster_labels)
# total_count = len(cluster_labels)
# percentages = (counts / total_count) * 100

# for number, percentage in enumerate(percentages, start=1):
#     print("{}: {:.2f}%".format(number, percentage))

## DBSCAN

In [35]:
# try DBSCAN
# for eps in range(10, 15):
#     print("\neps={}".format(eps))
#     dbscan = DBSCAN(eps=eps, min_samples=5)
#     labels = dbscan.fit_predict(train_df)
#     print("Number of clusters: {}".format(len(np.unique(labels))))
#     print("Cluster sizes: {}".format(np.bincount(labels + 1)))

## HDBSCAN

In [36]:
# min_cluster_size_values = [5, 10, 15]
# min_samples_values = [1, 2, 3]

# for min_cluster_size in min_cluster_size_values:
#     for min_samples in min_samples_values:
#         print("min_cluster_size={}, min_samples={}".format(min_cluster_size, min_samples))
        
#         hdbscan_clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, 
#                                             min_samples=min_samples)
        
#         labels = hdbscan_clusterer.fit_predict(train_df)
        
#         num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
#         cluster_sizes = [list(labels).count(i) for i in range(num_clusters)]
#         print("Number of clusters:", num_clusters)
#         print("Cluster sizes:", cluster_sizes)
#         print()

## SK FCM

In [37]:
# Convert merged_df to an FDataGrid object
fdata = skfda.FDataGrid(merged_df.values)

n_clusters=5
fcm = skfda.ml.clustering.FuzzyCMeans(n_clusters=n_clusters, fuzzifier=1.1, max_iter=1000, random_state=42)
# fcm = skfda.ml.clustering.FuzzyCMeans(random_state=42)
fcm.fit(fdata)

# Get the cluster labels
cluster_membership = fcm.labels_

In [38]:
#################### cluster percentages #############
cluster_percentages = []
for cluster in range(n_clusters):
    percentage = np.sum(cluster_membership == cluster) / len(cluster_membership) * 100
    cluster_percentages.append(percentage)

# Print the cluster percentages
for cluster, percentage in enumerate(cluster_percentages):
    print(f"Cluster {cluster} Percentage: {percentage:.2f}%")

################## Get sample stores ##################
cluster_samples_dict = {}

# Select 10 random samples from each cluster
for cluster in range(n_clusters):
    cluster_indices = np.where(cluster_membership == cluster)[0]
    if len(cluster_indices) >= 50:
        np.random.seed(42)
        cluster_samples = np.random.choice(cluster_indices, size=50, replace=False)
    else:
        cluster_samples = cluster_indices
    
    # Store the samples in different cluster list
    cluster_samples_dict[cluster] = merged_df.index[cluster_samples].tolist()

#Print
# for cluster, samples in cluster_samples_dict.items():
#     print(f"Cluster {cluster} Samples:")
#     for sample in samples:
#         print("Store:", sample)
#     print()

################## Get sample stores' long and latt ##############3#####
cluster_coordinates = {}

for cluster, samples in cluster_samples_dict.items():
    cluster_coordinates[cluster] = []
    for sample in samples:
        # Get the longitude and latitude
        longitude = stores.loc[sample, "longitude"]
        latitude = stores.loc[sample, "latitude"]
        
        # Append the [longitude, latitude] pair to the corresponding cluster list
        cluster_coordinates[cluster].append([longitude, latitude])

cluster_coordinates
# Print
# for cluster, coordinates_list in cluster_coordinates.items():
#     print(f"Cluster {cluster} Coordinates:")
#     for coordinates in coordinates_list:
#         print("Coordinates:", coordinates)
#     print()




Cluster 0 Percentage: 31.50%
Cluster 1 Percentage: 24.97%
Cluster 2 Percentage: 1.47%
Cluster 3 Percentage: 25.82%
Cluster 4 Percentage: 16.23%


{0: [[-82.3210157894299, 40.863749175856015],
  [-85.18023886238797, 38.368348290731944],
  [-79.42920310586047, 35.18018280563528],
  [-85.76160175750236, 33.81691712240363],
  [-159.3682617911413, 21.977888579995863],
  [-92.89989713054146, 41.397483583385366],
  [-96.38458959835582, 35.087116202442985],
  [-82.125936, 38.86967],
  [-92.26076917086893, 30.68319360616485],
  [-95.988477, 30.087406],
  [-93.57809339911816, 36.36802121082206],
  [-95.854075, 32.200557],
  [-86.48325406502836, 41.46714858852838],
  [-92.0047088769148, 41.69163295474276],
  [-93.85070597598342, 33.92816436941126],
  [-81.63832059694364, 41.005400078984216],
  [-110.791416, 43.469258],
  [-93.86290065378508, 32.45356410928552],
  [-82.847341, 36.145236],
  [-116.52701261185857, 33.80015607475938],
  [-80.61253095214497, 40.4518369966509],
  [-97.639947, 26.14847],
  [-85.44556277179616, 39.63008153381752],
  [-91.3231545590576, 38.40557168151618],
  [-80.62072, 36.498657],
  [-97.00972492867533, 38.3479438

## FCM

In [39]:
# # hyperparameter tune
# k_values = [5, 6, 7]
# m_values = [1.1, 1.2, 1.3, 1.5]

# best_score = -1
# best_k = None
# best_m = None

# # Grid search over K and m
# for k in k_values:
#     for m in m_values:
#         # FCM
#         cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(
#             merged_df.T, k, m, error=0.005, maxiter=1000, init=None, seed=42
#         )
        
#         # cluster labels
#         cluster_membership = np.argmax(u, axis=0)
        
#         # Calculate silhouette score
#         score = silhouette_score(merged_df, cluster_membership)
        
#         # best score
#         if score > best_score:
#             best_score = score
#             best_k = k
#             best_m = m

# # Print the best parameters and Silhouette score
# print("Best Parameters:")
# print("K:", best_k)
# print("m:", best_m)
# print("Best Silhouette Score:", best_score)

In [40]:
# the number of clusters (K) and the fuzziness parameter (m)
# (increases the fuzziness and allows for more overlapping clusters)
n_clusters = 5
m = 1.1

# FCM
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(
    merged_df.T, n_clusters, m, error=0.005, maxiter=1000, init=None, seed=42
)

# cluster labels
cluster_membership = np.argmax(u, axis=0)

In [41]:
############# cluster percentages #####################
cluster_percentages = []
for cluster in range(n_clusters):
    percentage = np.sum(cluster_membership == cluster) / len(cluster_membership) * 100
    cluster_percentages.append(percentage)

# Print the cluster percentages
for cluster, percentage in enumerate(cluster_percentages):
    print(f"Cluster {cluster} Percentage: {percentage:.2f}%")
cluster_samples_dict = {}

##################### sample stores #################
for cluster in range(n_clusters):
    cluster_indices = np.where(cluster_membership == cluster)[0]
    if len(cluster_indices) >= 10:
        np.random.seed(42)
        cluster_samples = np.random.choice(cluster_indices, size=10, replace=False)
    else:
        cluster_samples = cluster_indices
    
    # Store the samples in different cluster list
    cluster_samples_dict[cluster] = merged_df.index[cluster_samples].tolist()

#Print
# for cluster, samples in cluster_samples_dict.items():
#     print(f"Cluster {cluster} Samples:")
#     for sample in samples:
#         print("Store:", sample)
#     print()

############ stores' long and latt ########################
cluster_coordinates = {}

for cluster, samples in cluster_samples_dict.items():
    cluster_coordinates[cluster] = []
    for sample in samples:
        # Get the longitude and latitude
        longitude = stores.loc[sample, "longitude"]
        latitude = stores.loc[sample, "latitude"]
        
        # Append the [longitude, latitude] pair to the corresponding cluster list
        cluster_coordinates[cluster].append([longitude, latitude])

cluster_coordinates
# Print
# for cluster, coordinates_list in cluster_coordinates.items():
#     print(f"Cluster {cluster} Coordinates:")
#     for coordinates in coordinates_list:
#         print("Coordinates:", coordinates)
#     print()


Cluster 0 Percentage: 31.97%
Cluster 1 Percentage: 24.62%
Cluster 2 Percentage: 25.15%
Cluster 3 Percentage: 16.76%
Cluster 4 Percentage: 1.50%


{0: [[-75.87929011233992, 43.9688585943187],
  [-89.01706838079929, 34.25069996232895],
  [-87.7168340543012, 34.51354805344206],
  [-80.10063854263839, 35.76931735968024],
  [-77.36994150936053, 40.58424691652046],
  [-82.35530241155423, 31.78007154063636],
  [-97.468858, 27.798254],
  [-89.029338, 34.499313],
  [-100.402393, 32.472794],
  [-79.43908562005201, 36.094332161417405]],
 1: [[-122.21148457180873, 37.77438584799722],
  [-112.16668380374202, 33.59687569683549],
  [-97.61151877545008, 35.54696567606543],
  [-121.32580482859464, 38.65020717037968],
  [-90.06160530535014, 29.99893389597656],
  [-85.62432497802962, 42.84198134879875],
  [-87.70445196645196, 41.84035475833942],
  [-90.05339650597905, 30.028192215132155],
  [-122.49387989661425, 45.50556916570039],
  [-91.1361199443637, 30.424561036547196]],
 2: [[-75.5096016945444, 40.526556884002936],
  [-112.162327, 41.711005],
  [-81.90379349052002, 41.387667931632535],
  [-84.86784557210511, 32.53968928540907],
  [-77.3260684

## GMM

The Silhouette Score ranges from -1 to 1, where a higher score indicates better clustering structure. A score close to 1 suggests well-separated clusters, while a score close to -1 indicates that samples might have been assigned to the wrong clusters.



The calinski_harabasz_score() function calculates the Calinski-Harabasz Index, which measures the ratio between the within-cluster dispersion and between-cluster dispersion. Higher values indicate better-defined and more separated clusters.


The davies_bouldin_score() function calculates the Davies-Bouldin Index, which evaluates the average similarity between each cluster and its most similar cluster, taking into account both the within-cluster and between-cluster distances. Lower values indicate better clustering results.

In [42]:
# # tune
# k_values = [5, 6, 7]
# covariance_types = ['full', 'tied', 'diag', 'spherical']
# reg_params = [0.001, 0.1, 1, 10]

# best_score = -1
# best_k = None
# best_covariance_type = None
# # best_reg_param = None

# # Grid search
# for k in k_values:
#     for covariance_type in covariance_types:
#         for reg_param in reg_params:
#             # GMM
#             gmm = GaussianMixture(n_components=k, covariance_type=covariance_type,
#                                   reg_covar=reg_param, 
#                                   random_state=42)
#             gmm.fit(merged_df)
            
#             # cluster labels
#             cluster_membership = gmm.predict(merged_df)
            
#             # silhouette score
#             score = silhouette_score(merged_df, cluster_membership)
#             # Calculate Calinski-Harabasz Index
#             # score = calinski_harabasz_score(merged_df, cluster_membership)
#             # Calculate Davies-Bouldin Index
#             # score = davies_bouldin_score(merged_df, cluster_membership)

#             # best
#             if score > best_score:
#                 best_score = score
#                 best_k = k
#                 best_covariance_type = covariance_type
#                 best_reg_param = reg_param
# 
# # Print the best hyperparameters and score
# print("Best Hyperparameters:")
# print("K:", best_k)
# print("Covariance Type:", best_covariance_type)
# print("Regularization Parameter:", best_reg_param)
# print("Best Score:", best_score)

In [43]:
# best GMM
best_k = 5
best_covariance_type = "full"
best_reg_param = 0.008
best_gmm = GaussianMixture(n_components=best_k, 
                           covariance_type=best_covariance_type,
                           reg_covar=best_reg_param,
                           random_state=42)
best_gmm.fit(merged_df)

# cluster labels
cluster_membership = best_gmm.predict(merged_df)


In [44]:
############### cluster percentages ###################
cluster_percentages = []
for cluster in range(best_k):
    percentage = np.sum(cluster_membership == cluster) / len(cluster_membership) * 100
    cluster_percentages.append(percentage)

# cluster percentages
for cluster, percentage in enumerate(cluster_percentages):
    print(f"Cluster {cluster} Percentage: {percentage:.2f}%")

# Get the cluster labels
cluster_membership = best_gmm.predict(merged_df)

cluster_samples_dict = {}

##################### sample stores ####################
for cluster in range(n_clusters):
    cluster_indices = np.where(cluster_membership == cluster)[0]
    if len(cluster_indices) >= 10:
        np.random.seed(42)
        cluster_samples = np.random.choice(cluster_indices, size=10, replace=False)
    else:
        cluster_samples = cluster_indices
    
    # Store the samples in different cluster list
    cluster_samples_dict[cluster] = merged_df.index[cluster_samples].tolist()

#Print
# for cluster, samples in cluster_samples_dict.items():
#     print(f"Cluster {cluster} Samples:")
#     for sample in samples:
#         print("Store:", sample)
#     print()

############## long and latt ########################
cluster_coordinates = {}

for cluster, samples in cluster_samples_dict.items():
    cluster_coordinates[cluster] = []
    for sample in samples:
        # Get the longitude and latitude
        longitude = stores.loc[sample, "longitude"]
        latitude = stores.loc[sample, "latitude"]
        
        # Append the [longitude, latitude] pair to the corresponding cluster list
        cluster_coordinates[cluster].append([longitude, latitude])

cluster_coordinates
# Print
# for cluster, coordinates_list in cluster_coordinates.items():
#     print(f"Cluster {cluster} Coordinates:")
#     for coordinates in coordinates_list:
#         print("Coordinates:", coordinates)
#     print()

Cluster 0 Percentage: 1.95%
Cluster 1 Percentage: 10.27%
Cluster 2 Percentage: 36.25%
Cluster 3 Percentage: 20.99%
Cluster 4 Percentage: 30.54%


{0: [[-87.62889844071469, 41.87673649632343],
  [-73.78922166580193, 40.766165429617374],
  [-73.90246480662357, 40.74093281046426],
  [-122.347834, 47.615102],
  [-73.98500825252522, 40.59729237859677],
  [-73.85817726441854, 40.91425055057032],
  [-73.86011618486515, 40.88786219595651],
  [-73.94920018055247, 40.65103641239335],
  [-73.97335542738496, 40.752692311078235],
  [-73.95288266446354, 40.59941161821086]],
 1: [[-80.29426567237309, 25.93247364698191],
  [-77.001623501121, 38.958589550180946],
  [-76.84740133074392, 38.922019110393016],
  [-80.07997418958907, 26.386843146894883],
  [-76.288975, 36.846017],
  [-117.15710047053098, 32.80161166814661],
  [-80.1896501312866, 25.777167605961527],
  [-118.39470992282666, 33.9595235020714],
  [-87.6688858991975, 41.86643438742046],
  [-111.89264790256502, 33.582839696427015]],
 2: [[-82.67552698119415, 30.178390403467848],
  [-86.31318124768376, 34.336824927668395],
  [-89.36928224458526, 41.111345763149146],
  [-101.751122, 34.1928

## AgglomerativeClustering

In [45]:
# number of clusters (K)
n_clusters = 5

# Agglomerative Clustering model
agglomerative = AgglomerativeClustering(n_clusters=n_clusters, linkage="ward")
agglomerative.fit(merged_df)

# cluster labels
cluster_membership = agglomerative.labels_

NameError: name 'AgglomerativeClustering' is not defined

In [ ]:
#################### cluster percentages ######################
cluster_percentages = []
for cluster in range(best_k):
    percentage = np.sum(cluster_membership == cluster) / len(cluster_membership) * 100
    cluster_percentages.append(percentage)

for cluster, percentage in enumerate(cluster_percentages):
    print(f"Cluster {cluster} Percentage: {percentage:.2f}%")

cluster_samples_dict = {}

########################### sample stores ###########################
for cluster in range(n_clusters):
    cluster_indices = np.where(cluster_membership == cluster)[0]
    if len(cluster_indices) >= 10:
        np.random.seed(42)
        cluster_samples = np.random.choice(cluster_indices, size=10, replace=False)
    else:
        cluster_samples = cluster_indices
    
    # Store the samples in different cluster list
    cluster_samples_dict[cluster] = merged_df.index[cluster_samples].tolist()

#Print
# for cluster, samples in cluster_samples_dict.items():
#     print(f"Cluster {cluster} Samples:")
#     for sample in samples:
#         print("Store:", sample)
#     print()

########################### long and latt ###########################
cluster_coordinates = {}

for cluster, samples in cluster_samples_dict.items():
    cluster_coordinates[cluster] = []
    for sample in samples:
        # Get the longitude and latitude
        longitude = stores.loc[sample, "longitude"]
        latitude = stores.loc[sample, "latitude"]
        
        # Append the [longitude, latitude] pair to the corresponding cluster list
        cluster_coordinates[cluster].append([longitude, latitude])

cluster_coordinates
# Print
# for cluster, coordinates_list in cluster_coordinates.items():
#     print(f"Cluster {cluster} Coordinates:")
#     for coordinates in coordinates_list:
#         print("Coordinates:", coordinates)
#     print()

